# FPL with genetic algorithms

Todo
 - pycall to get data
 - package environment

In [1]:
using CSV
using StatsBase
using Statistics
using DataFrames

In [2]:
push!(LOAD_PATH, "/home/duncan/Documents/genetic_fpl_julia")
using FplFuncs

┌ Info: Precompiling FplFuncs [top-level]
└ @ Base loading.jl:1278


In [3]:
df = DataFrame(CSV.File("./data.csv"))
df["fitness"] = df.ep_next
# df["fitness"] = fitness.(df.points_per_game, df.form)
first(df, 3)

,first_name,second_name,team,team_code,id,element_type,now_cost
,String,String,Int64,Int64,Int64,Int64,Float64
1,Mesut,Özil,1,3,1,3,6.8
2,Sokratis,Papastathopoulos,1,3,2,2,4.9
3,David,Luiz Moreira Marinho,1,3,3,2,5.5


In [4]:
id_to_player, id_to_cost, id_to_element_type, id_to_team, id_to_fitness = FplFuncs.fpl_get_dicts(df);

In [5]:
typeof(id_to_element_type)

Dict{Any,Any}

In [6]:
formation = FplFuncs.fpl_generate_formation()

(1, 5, 4, 1)

In [7]:
initial_squad = FplFuncs.fpl_generate_squad_initial(df)

15-element Array{Int64,1}:
  32
 252
 196
 197
 270
  47
 152
 500
 411
 198
 537
 446
 416
 506
 340

In [8]:
team_from_squad = FplFuncs.fpl_generate_team_from_squad(initial_squad, id_to_element_type)

15-element Array{Int64,1}:
 252
 196
 197
 152
  47
 500
 446
 198
 411
 340
 506
  32
 270
 537
 416

In [9]:
FplFuncs.fpl_print_squad_names(initial_squad, id_to_player)

32 Steer
252 Ramses Becker
196 Douglas
197 Ayling
270 Walker
47 Guilbert
152 Coleman
500 Havertz
411 Phillips
198 Klich
537 Bernabé
446 Diangana
416 Zohore
506 Wilson
340 Apolinário de Lira


In [10]:
FplFuncs.fpl_squad_cost(initial_squad, id_to_cost)

80.10000000000001

In [11]:
FplFuncs.fpl_team_fitness(initial_squad, id_to_fitness)

20.5

In [12]:
FplFuncs.fpl_most_from_one_squad(initial_squad, id_to_team)

3

In [13]:
FplFuncs.fpl_validate_player_types(initial_squad, id_to_element_type)

false

In [14]:
FplFuncs.fpl_players_unique(initial_squad)

true

In [15]:
FplFuncs.fpl_get_player_types(initial_squad, id_to_element_type)

Squad types: [2.0, 5.0, 5.0, 3.0]


4-element Array{Float64,1}:
 2.0
 5.0
 4.0
 0.0

In [16]:
FplFuncs.fpl_print_player_types(initial_squad, id_to_element_type)

Squad types: [2.0, 5.0, 5.0, 3.0]
[2.0, 5.0, 4.0, 0.0]


In [17]:
FplFuncs.fpl_validate_squad(initial_squad, id_to_cost, id_to_team, id_to_element_type)

false

In [18]:
FplFuncs.fpl_iterate_valid_team(df,id_to_element_type,id_to_cost, id_to_team)

15-element Array{Int64,1}:
 350
  77
 545
 139
 587
 385
 149
  98
 296
  22
 268
 128
 378
  46
  86

In [36]:
pool_size = 200
generations = 1000
early_stop_lim = 200

pool = zeros(Int64, (pool_size, 15))
fitnesses = zeros(pool_size);
max_pool = zero(pool)
max_fitnesses = zero(fitnesses)

created = 0
while created < pool_size
    squad = FplFuncs.fpl_iterate_valid_team(df,
    id_to_element_type,
    id_to_cost,
    id_to_team
    )
    pool[created+1, :] = squad
    fit = FplFuncs.fpl_team_fitness(squad, id_to_fitness)
    fitnesses[created+1] = fit
    created += 1
end

max_fitness = 0
early_stop_count = 0

for generation in collect(1:generations)
    println("Generation: ", generation)
    new_pool = zero(pool)
    new_fitnesses = zero(fitnesses)
    parents, parent_fitness = FplFuncs.fpl_select_best_rows(pool,
        fitnesses,
        50
    )
    created=0
    while created < pool_size
        new_squad = FplFuncs.fpl_iterate_breed(parents,
            df,
            id_to_cost,
            id_to_team,
            id_to_element_type
        )
        new_pool[created+1, :] = new_squad
        new_fitnesses[created+1] = FplFuncs.fpl_team_fitness(new_squad,
            id_to_fitness
        )
        created += 1
    end
    pool = new_pool
    fitnesses = new_fitnesses
    println("Max fitness: ", maximum(fitnesses))
    if maximum(fitnesses) > max_fitness
        max_fitness = maximum(fitnesses)
        max_pool = pool
        max_fitnesses = fitnesses
        early_stop_count = 0
    elseif early_stop_count <= early_stop_lim
        early_stop_count += 1
    else
        println("Early stopping")
        break
    end
end

Generation: 1
Max fitness: 43.72500000000001
Generation: 2
Max fitness: 42.375
Generation: 3
Max fitness: 41.074999999999996
Generation: 4
Max fitness: 44.775000000000006
Generation: 5
Max fitness: 47.550000000000004
Generation: 6
Max fitness: 43.62499999999999
Generation: 7
Max fitness: 46.349999999999994
Generation: 8
Max fitness: 49.875
Generation: 9
Max fitness: 48.15
Generation: 10
Max fitness: 50.050000000000004
Generation: 11
Max fitness: 50.025
Generation: 12
Max fitness: 50.225
Generation: 13
Max fitness: 52.225
Generation: 14
Max fitness: 50.1
Generation: 15
Max fitness: 49.6
Generation: 16
Max fitness: 51.824999999999996
Generation: 17
Max fitness: 52.55
Generation: 18
Max fitness: 51.824999999999996
Generation: 19
Max fitness: 51.824999999999996
Generation: 20
Max fitness: 52.574999999999996
Generation: 21
Max fitness: 52.925000000000004
Generation: 22
Max fitness: 52.25
Generation: 23
Max fitness: 53.824999999999996
Generation: 24
Max fitness: 52.49999999999999
Generation:

Generation: 201
Max fitness: 58.32499999999999
Generation: 202
Max fitness: 58.44999999999999
Generation: 203
Max fitness: 57.89999999999999
Generation: 204
Max fitness: 56.474999999999994
Generation: 205
Max fitness: 57.05
Generation: 206
Max fitness: 57.15
Generation: 207
Max fitness: 57.099999999999994
Generation: 208
Max fitness: 58.675000000000004
Generation: 209
Max fitness: 57.875
Generation: 210
Max fitness: 58.4
Generation: 211
Max fitness: 58.4
Generation: 212
Max fitness: 57.425000000000004
Generation: 213
Max fitness: 57.15
Generation: 214
Max fitness: 57.699999999999996
Generation: 215
Max fitness: 57.574999999999996
Generation: 216
Max fitness: 58.175000000000004
Generation: 217
Max fitness: 57.224999999999994
Generation: 218
Max fitness: 56.349999999999994
Generation: 219
Max fitness: 56.349999999999994
Generation: 220
Max fitness: 57.699999999999996
Generation: 221
Max fitness: 57.025
Generation: 222
Max fitness: 62.625
Generation: 223
Max fitness: 57.025
Generation: 22

Max fitness: 70.0
Generation: 410
Max fitness: 68.375
Generation: 411
Max fitness: 67.35000000000001
Generation: 412
Max fitness: 70.2
Generation: 413
Max fitness: 68.35000000000001
Generation: 414
Max fitness: 69.2
Generation: 415
Max fitness: 65.19999999999999
Generation: 416
Max fitness: 69.075
Generation: 417
Max fitness: 68.8
Generation: 418
Max fitness: 65.05
Generation: 419
Max fitness: 66.2
Generation: 420
Max fitness: 65.675
Generation: 421
Max fitness: 68.35000000000001
Generation: 422
Max fitness: 64.47500000000001
Generation: 423
Max fitness: 65.25
Generation: 424
Max fitness: 64.47500000000001
Generation: 425
Max fitness: 64.35000000000001
Generation: 426
Max fitness: 63.825
Generation: 427
Max fitness: 65.275
Generation: 428
Max fitness: 64.35000000000001
Generation: 429
Max fitness: 65.275
Generation: 430
Max fitness: 64.47500000000001
Generation: 431
Max fitness: 65.675
Generation: 432
Max fitness: 64.10000000000001
Generation: 433
Max fitness: 64.47500000000001
Generat

Max fitness: 64.675
Generation: 610
Max fitness: 65.2
Generation: 611
Max fitness: 65.2
Generation: 612
Max fitness: 64.65
Generation: 613
Max fitness: 65.2
Generation: 614
Max fitness: 66.39999999999999
Early stopping


In [37]:
best_team = max_pool[argmax(max_fitnesses), :]
FplFuncs.fpl_print_squad_names(best_team, id_to_player)

363 McCarthy
459 Coady
155 Digne
52 Soares de Paulo
232 Chilwell
46 Konsa Ngoyo
302 Borges Fernandes
254 Salah
338 Saint-Maximin
445 Bowen
164 Calvert-Lewin
431 Fabianski
102 Azpilicueta
444 Hugill
117 Werner


In [39]:
FplFuncs.fpl_team_fitness(best_team, id_to_fitness)

70.2

In [40]:
FplFuncs.fpl_squad_cost(best_team, id_to_cost)

98.8

In [41]:
typeof(best_team)

Array{Int64,1}

In [42]:
FplFuncs.fpl_print_player_types(best_team, id_to_element_type)

Squad types: [2.0, 5.0, 5.0, 3.0]
[1.0, 4.0, 5.0, 1.0]


#### Optimisation of squad

Change one/two players

In [27]:
# chosen_team = [12,
#     41,
#     498,
#     46,
#     110,
#     508,
#     445,
#     105,
#     141,
#     164,
#     224,
#     325,
#     435,
#     137,
#     506]
# print_squad_names(chosen_team)

In [43]:
# Fergus' team
chosen_team = [
    131,
    200,
    255,
    259,
    302,
    254,
    508,
    284,
    262,
    117,
    184,
    99,
    457,
    449,
    146
]
FplFuncs.fpl_print_squad_names(chosen_team, id_to_player)

131 McCarthy
200 Dallas
255 Robertson
259 Alexander-Arnold
302 Borges Fernandes
254 Salah
508 Rodríguez
284 Foden
262 Brewster
117 Werner
184 Mitrović
99 Peacock-Farrell
457 Doherty
449 Soucek
146 Mitchell


In [46]:
pool_size = 200
generations = 1000
early_stop_lim = 200
n_changes = 1

base_pool = zeros(Int64, (pool_size, 15))
fitnesses = zeros(pool_size);
max_pool = zero(pool)
max_fitnesses = zero(fitnesses)

for i in 1:pool_size
    base_pool[i, :] = chosen_team
end

fitnesses[:] .= FplFuncs.fpl_team_fitness(base_pool[1, :], id_to_fitness)

max_fitness = FplFuncs.fpl_team_fitness(base_pool[1, :], id_to_fitness)
early_stop_count = 0

for generation in collect(1:generations)
    println("Generation: ", generation)
    new_pool = zero(base_pool)
    new_fitnesses = zero(fitnesses)
    parents, parent_fitness = FplFuncs.fpl_select_best_rows(base_pool, fitnesses, 50)
    created = 0
    while created < pool_size
        new_squad = FplFuncs.fpl_iterate_breed(parents,
            df,
            id_to_cost,
            id_to_team,
            id_to_element_type,
            chosen_team,
            n_changes
        )
        new_pool[created+1, :] = new_squad
        new_fitnesses[created+1] = FplFuncs.fpl_team_fitness(new_squad,
        id_to_fitness
        )
        created += 1
    end
    base_pool = new_pool
    fitnesses = new_fitnesses
    println("Maximum fitness: ", maximum(fitnesses))
    if maximum(fitnesses) > max_fitness
        max_fitness = maximum(fitnesses)
        max_pool = base_pool
        max_fitnesses = fitnesses
        early_stop_count = 0
    elseif early_stop_count <= early_stop_lim
        early_stop_count += 1
    else
        println("Early stopping")
        break
    end
end

Generation: 1
Maximum fitness: 58.175
Generation: 2
Maximum fitness: 58.175
Generation: 3
Maximum fitness: 58.175
Generation: 4
Maximum fitness: 59.3
Generation: 5
Maximum fitness: 59.3
Generation: 6
Maximum fitness: 58.175
Generation: 7
Maximum fitness: 58.175
Generation: 8
Maximum fitness: 51.974999999999994
Generation: 9
Maximum fitness: 50.925
Generation: 10
Maximum fitness: 50.925
Generation: 11
Maximum fitness: 50.1
Generation: 12
Maximum fitness: 50.125
Generation: 13
Maximum fitness: 50.1
Generation: 14
Maximum fitness: 48.975
Generation: 15
Maximum fitness: 50.1
Generation: 16
Maximum fitness: 50.1
Generation: 17
Maximum fitness: 50.925
Generation: 18
Maximum fitness: 50.925
Generation: 19
Maximum fitness: 50.1
Generation: 20
Maximum fitness: 48.975
Generation: 21
Maximum fitness: 50.1
Generation: 22
Maximum fitness: 48.975
Generation: 23
Maximum fitness: 50.1
Generation: 24
Maximum fitness: 50.925
Generation: 25
Maximum fitness: 48.975
Generation: 26
Maximum fitness: 50.1
Gen

Maximum fitness: 49.175000000000004
Generation: 192
Maximum fitness: 51.125
Generation: 193
Maximum fitness: 50.300000000000004
Generation: 194
Maximum fitness: 51.125
Generation: 195
Maximum fitness: 51.125
Generation: 196
Maximum fitness: 50.300000000000004
Generation: 197
Maximum fitness: 49.175000000000004
Generation: 198
Maximum fitness: 48.875
Generation: 199
Maximum fitness: 58.175
Generation: 200
Maximum fitness: 51.125
Generation: 201
Maximum fitness: 50.300000000000004
Generation: 202
Maximum fitness: 51.125
Generation: 203
Maximum fitness: 51.125
Generation: 204
Maximum fitness: 50.300000000000004
Generation: 205
Maximum fitness: 51.125
Generation: 206
Maximum fitness: 51.974999999999994
Early stopping


In [47]:
FplFuncs.fpl_print_squad_names(chosen_team, id_to_player)
FplFuncs.fpl_team_fitness(chosen_team, id_to_fitness)

131 McCarthy
200 Dallas
255 Robertson
259 Alexander-Arnold
302 Borges Fernandes
254 Salah
508 Rodríguez
284 Foden
262 Brewster
117 Werner
184 Mitrović
99 Peacock-Farrell
457 Doherty
449 Soucek
146 Mitchell


53.474999999999994

In [48]:
best_team = max_pool[argmax(max_fitnesses), :]
FplFuncs.fpl_print_squad_names(best_team, id_to_player)

213 Meslier
200 Dallas
255 Robertson
259 Alexander-Arnold
302 Borges Fernandes
254 Salah
508 Rodríguez
284 Foden
262 Brewster
117 Werner
449 Soucek
99 Peacock-Farrell
457 Doherty
184 Mitrović
146 Mitchell


In [49]:
FplFuncs.fpl_team_fitness(max_pool[argmax(max_fitnesses), :], id_to_fitness)

59.3